# Defining Fault Sampling Approaches in fmdtools

Fault Sampling is used to evaluate the overall resilience of a system to a set of faults and the corresponding risks associated with these faults. There is no single best way to define the set of scenarios to evaluate resilience with, because a given resilience analysis may need more or less detail, support more or less computational time, or be interested in specific scenario types of interest.

Thus, there are a number of use-cases supported by fmdtools for different sampling models. This document will demonstrate and showcase a few of them.

```
Copyright © 2024, United States Government, as represented by the Administrator of the National Aeronautics and Space Administration. All rights reserved.

The “"Fault Model Design tools - fmdtools version 2"” software is licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```


In [1]:
from fmdtools.sim.sample import SampleApproach, FaultSample, FaultDomain
from fmdtools.analyze.tabulate import FMEA, Comparison
import fmdtools.sim.propagate as prop
import numpy as np

### Basics

Fault sampling involves:
- Defining faults and fault models for each function/component/flows of the model is done using the 'Mode' Class
- Defining a fault sampling approach, using the: 
    - `SampleApproach` class, or
    -  `FaultDomain` and `FaultSample` calsses directly
- Propagating faults through the model (using the `propagate.faultsample` method)
Before proceeding, it can be helpful to look through their respective documentation.

### Model Setup

Consider the rover model in `rover_model`

In [2]:
import inspect
from rover_model import Power, PowerMode

ModuleNotFoundError: No module named 'rover_model'

This rover has a Power function:

In [3]:
print(inspect.getsource(Power))

Which contains the mode `PowerMode`:

In [4]:
print(inspect.getsource(PowerMode))

The class variable `fm_args` specifies that there are two possible modes to inject, "no_charge", and "shortn", with no more information given for each mode. More information has been added in the tuples of the dictionary, including:
- rate
- repair cost
- phase dictionary
The phase dictionary is important because it specifies that this mode is to occur in a given phase. In this case, `no_charge` is supposed to only occur during the `standby` phase while `short` is only supposed to occur during the `supply` phase. In this `Mode`, these phases correspond to the operational modes (`opermodes`), but they may correspond to other operational modes also.

All of these fields are optional, but they do help us develop a more informed statistical sample of the fault modes.

### Setting up a FaultSample

Sampling using `FaultSample` first requires creating a `FaultDomain` to sample from. These faultdomains can be created from both models and individual functions:

In [5]:
fd_power = FaultDomain(Power())
fd_power.add_all()
fd_power

In [6]:
from rover_model import Rover
fd_rvr = FaultDomain(Rover())
fd_rvr.add_all()
fd_rvr

Note that there are several methods in FaultDomain which let us specify the list of faults we want to sample from, e.g.:

In [7]:
fd_short = FaultDomain(Rover())
fd_short.add_all_modes("short")
fd_short

or:

In [8]:
fd_pwr = FaultDomain(Rover())
fd_pwr.add_all_fxn_modes("power")
fd_pwr

We can then sample this domain using a `FaultSample`:

In [9]:
fs_pwr = FaultSample(fd_pwr, def_mdl_phasemap=False)

Note that FaultSamples have two main variables: faultdomain and phasemap. A PhaseMap is essentially a dictionary of phases to sample from.

In the above case, we don't want to use phase information to form the sample, so we don't provide any and we set `def_mdl_phasemap=False`, since this would get phase information from the model.

To add scenarios to the `FaultSample`, we can then use one of the `add` methods:

In [10]:
fs_pwr.add_fault_times([2,5,10])
fs_pwr

As shown, this adds the list of faults in the faultdomain over the given times.

Note the underlying rate information in these scenarios is all the same:

In [11]:
fs_pwr.scenarios()

But we know better than this--some of the faults should have zero rate if they are going to occur in phases they don't apply to!

If we want to sample the given phases for the sample, we can additionally pass a phasemap generated by first running the model in the nominal state:

In [12]:
res, hist = prop.nominal(Rover())

We can then get phase information from this history using the `fmdtools.analyze.phases.from_hist`:

In [13]:
from fmdtools.analyze.phases import from_hist, phaseplot
phases = from_hist(hist)
phases

Which can be visualized using:

In [14]:
fig = phaseplot(phases)

The `PhaseMap` for `Power` is here in `power`:

In [15]:
phases['power']

Which we can use to create a `FaultSample` which only samples the phases corresponding to the information given in `Mode`:

In [16]:
fs_pwr = FaultSample(fd_pwr, phasemap=phases['power'])
fs_pwr.add_fault_phases()
fs_pwr

If we look at the rate information, however:

In [17]:
fs_pwr.scenarios()

The rate for scenarios outside the phases is zero!

We can remove these scenarios using `FaultSample.prune_scenarios`:

In [18]:
fs_pwr.prune_scenarios("rate", np.greater, 0.0)
fs_pwr

In [19]:
fs_pwr.scenarios()

As shown, now the only scenarios in the `FaultSample` are ones which have nonzero rate.

In [20]:
assert all([scen.rate > 0 for scen in fs_pwr.scenarios()])

To enable multiple samples to be generated for different faultdomains accross the model, we can use `SampleApproach`, e.g.:

In [21]:
phases

In [22]:
sa = SampleApproach(Rover(), phasemaps=phases)
# adding fault domains
sa.add_faultdomain("drive", "fault", "drive", "hmode_1")
sa.add_faultdomain("plan_path", "all_fxn_modes", "plan_path")
sa.add_faultdomain("power", "all_fxn_modes", "power")
sa.add_faultsample("drive", "fault_phases", "drive", phasemap="plan_path")
sa.add_faultsample("plan_path", "fault_phases", "plan_path", phasemap="plan_path")
sa.add_faultsample("power", "fault_phases", "power", phasemap="power")
sa.prune_scenarios()

In [23]:
sa

In [24]:
sa.scenarios()

This is mostly useful when we would like to sample different functions in a model differently than others (e.g., using different phases), but still want to propagate the scenarios together as a part of a single sample.

### Propagating Faults

Given the `FaultSample` approach, faults can then be propagated through the model to get results. Note that these faults can be sampled in parallel if desired using a user-provided pool (see the parallel pool tutorial in the `\pump example` folder).

In [25]:
res, hist = prop.fault_sample(Rover(), fs_pwr)

In [26]:
res

These responses can be visualized over the given faults:

In [27]:
fmea = FMEA(res, fs_pwr, metrics = ["end_dist", "line_dist", "tot_deviation"])
fmea.as_table()

In [28]:
fmea.as_plots(cols=2)

Or over time/any other variable:

In [29]:
comp = Comparison(res, fs_pwr, metrics = ["end_dist", "line_dist", "tot_deviation"], factors =['time'])
comp.as_table()

In [30]:
comp.as_plots(cols=2)